In [2]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

In [8]:
def normalize_reviews(filepath):
  '''
  1. Drop duplicated rows
  2. Normalize dates to only the year
  3. Normalize ratings to 10
  '''
  
  df = pd.read_excel(filepath)
  
  # Drop duplicated rows
  df.drop_duplicates(keep='first', inplace=True)
  
  '''
  1. Normalize dates to only year
    - fix_date_week | fix_date_month | fix_date_year
      - Takes current date and minus off respective week, month or year
  '''
  
  def fix_date_week(week):
    date = datetime.datetime.now()
    date = date.date()
    newdate = date - relativedelta(weeks=week)
    return newdate
  
  def fix_date_month(month):
    date = datetime.datetime.now()
    date = date.date()
    newdate = date - relativedelta(months=month)
    return newdate
  
  def fix_date_year(year):
    date = datetime.datetime.now()
    date = date.date()
    newdate = date - relativedelta(years=year)
    return newdate
  
  # Replace date strings with proper year
  # Only up to 10 years
  replace_date = {'seminggu lalu': fix_date_week(1),
                  '2 minggu lalu': fix_date_week(2),
                  '3 minggu lalu': fix_date_week(3),
                  'sebulan lalu': fix_date_month(1),
                  '2 bulan lalu': fix_date_month(2),
                  '3 bulan lalu': fix_date_month(3),
                  '4 bulan lalu': fix_date_month(4),
                  '5 bulan lalu': fix_date_month(5),
                  '6 bulan lalu': fix_date_month(6),
                  '7 bulan lalu': fix_date_month(7),
                  '8 bulan lalu': fix_date_month(8),
                  '9 bulan lalu': fix_date_month(9),
                  '10 bulan lalu': fix_date_month(10),
                  '11 bulan lalu': fix_date_month(11),
                  'setahun lalu': fix_date_year(1),
                  '2 tahun lalu': fix_date_year(2),
                  '3 tahun lalu': fix_date_year(3),
                  '4 tahun lalu': fix_date_year(4),
                  '5 tahun lalu': fix_date_year(5),
                  '6 tahun lalu': fix_date_year(6),
                  '7 tahun lalu': fix_date_year(7),
                  '8 tahun lalu': fix_date_year(8),
                  '9 tahun lalu': fix_date_year(9),
                  '10 tahun lalu': fix_date_year(10),}
  
  # Split dataframe with proper datetime format
  df1 = df[df['date'].apply(lambda x: isinstance(x, datetime.date))]
  # Split dataframe with date strings
  df2 = df[df['date'].apply(lambda x: not isinstance(x, datetime.date))]
  # Replace date strings with proper year
  df2 = df2.replace({"date": replace_date})
  
  # Only extract year
  df1['date'] = pd.DatetimeIndex(df1['date']).year
  df2['date'] = pd.DatetimeIndex(df2['date']).year
  
  # Concatenate fixed year
  df = pd.concat([df1, df2], ignore_index=True)
  
  '''
  2. Normalize ratings to 10
  '''
  
  # Replace Klook ratings with 'Baik' to 8 and 'Sangat Direkomendasikan' to 10
  replace_rating = {'Baik': '8',
                  'Sangat Direkomendasikan': '10'}
  df = df.replace({'rating': replace_rating})
  
  # Mask for TripAdvisor ratings
  mask = (df['source'] == 'tripadvisor')
  # Retrieve index 0 string of rating and multiply by 2
  df.loc[mask, 'rating'] = (df.loc[mask, 'rating'].str[0].astype(int))*2
  
  # Mask for Google Review ratings
  mask2 = (df['source'] == 'google_reviews')
  # Retrieve index 0 string of rating and multiply by 2
  df.loc[mask2, 'rating'] = (df.loc[mask2, 'rating'].str[14].astype(int))*2
  
  # Export output 
  attraction = df['attraction'][0]
  df.to_excel(f'output/normalize_data/{attraction}_reviews_normalized.xlsx', index=False)

In [10]:
normalize_reviews("./raw_data/uss_master_copy.xlsx")

C:\Users\Luqman\AppData\Local\Temp\ipykernel_11224\243304745.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['date'] = pd.DatetimeIndex(df1['date']).year
